In [1]:
import pandas as pd,numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,recall_score,precision_score
from sklearn.linear_model import LogisticRegression
import plotly.express as px
from datetime import datetime
import time, random
import xgboost as xgb
#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [0]:
train = pd.read_csv("https://raw.githubusercontent.com/Layden23/bet369/master/Datasets/train3.csv", header = 0)

In [3]:
len(train["MatchID"].unique())

4356

In [4]:
train.isna().sum()

MatchID                    18
LeagueID                   18
LeagueName                 18
LeagueFullname             18
LeagueAbbreviation         18
CountryID                  18
CountryName                18
CountryAbbreviation        18
HomeTeamID                 18
HomeTeamName               18
GuestTeamID                18
GuestTeamName              18
HostHandicapFT            513
ExpectedGoalsFT           505
BettingCuoteFT         185933
HostHandicapHT            975
ExpectedGoalsHT           956
BettingCuoteHT         186606
HostGoalsFT                18
GuestGoalsFT               18
HostCornersFT              18
GuestCornersFT             18
HostYCFT                   18
GuestYCFT                  18
HostRCFT                   18
GuestRCFT                  18
HostAttacksFT              18
GuestAttacksFT             18
HostDAttacksFT             18
GuestDAttacksFT            18
HostonTargetFT             18
GuestonTargetFT            18
HostoffTargetFT            18
GuestoffTa

In [0]:
train = train.drop(train[train["MatchID"] == "MatchID"].index)

In [0]:
#Remove Strange Matches
train = train.loc[(train['HostPossessionFT'] != "0") & (train['GuestPossessionFT'] != "0")]
#Fix Goal Problem
train["Goal"] = pd.to_numeric(train["HostGoalsFT"]) + pd.to_numeric(train["GuestGoalsFT"])

In [7]:
train[train["Status"] == "HT"].iloc[:,-20:].sort_values(by = "Goal", ascending = False)

,HostGoalsFT,GuestGoalsFT,HostCornersFT,GuestCornersFT,HostYCFT,GuestYCFT,HostRCFT,GuestRCFT,HostAttacksFT,GuestAttacksFT,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
252672,10,0,8,0,0,1,0,0,62,41,54,1,20,0,4,0,69,31,HT,10.0
252104,10,0,8,0,0,1,0,0,62,41,54,1,20,0,4,0,69,31,HT,10.0
252364,10,0,8,0,0,1,0,0,62,41,54,1,20,0,4,0,69,31,HT,10.0
253175,10,0,8,0,0,1,0,0,62,41,54,1,20,0,4,0,69,31,HT,10.0
252978,10,0,8,0,0,1,0,0,62,41,54,1,20,0,4,0,69,31,HT,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187835,0,0,6,0,0,2,0,0,61,37,43,17,2,2,4,2,59,41,HT,0.0
187840,0,0,2,2,1,1,0,0,48,37,23,21,5,1,5,3,53,47,HT,0.0
187912,0,0,1,2,0,0,0,0,31,60,9,38,0,3,1,4,37,63,HT,0.0
187913,0,0,6,0,0,2,0,0,61,37,43,17,2,2,4,2,59,41,HT,0.0


In [0]:
s = 203

In [0]:
#Remove columns with lots of missing values
del train["BettingCuoteHT"]
del train["BettingCuoteFT"]
del train["HostGoalsFT"]
del train["GuestGoalsFT"]

In [0]:
#Creating train with matches that score in the first half
# train.Status = train.Status.replace(["HT","FT"],[45,90])
train = train.drop(train[train["Status"] == "NS"].index)
train = train.drop(train[train["Status"] == "HT"].index)
train = train.drop(train[train["Status"] == "FT"].index)
#Transform variables to numeric
for i in train.iloc[:,12:].columns:
    train[i] = pd.to_numeric(train[i])
#Creating train with matches that don't score in the first half
train0 = train[(train["Goal"] == 0) & (train["Status"] <= 45)]
random.seed(69)
train0 = train0.groupby('MatchID').apply(lambda x: x.sample(1))
random.seed(70)
train00 = train[(train["Goal"] == 0) & (train["Status"] <= 45)]
train00 = train00.groupby('MatchID').apply(lambda x: x.sample(1))
random.seed(619)
train000 = train[(train["Goal"] == 0) & (train["Status"] <= 45)]
train000 = train000.groupby('MatchID').apply(lambda x: x.sample(1))

In [11]:
train0

,,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,GuestTeamID,GuestTeamName,HostHandicapFT,ExpectedGoalsFT,HostHandicapHT,ExpectedGoalsHT,HostCornersFT,GuestCornersFT,HostYCFT,GuestYCFT,HostRCFT,GuestRCFT,HostAttacksFT,GuestAttacksFT,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
MatchID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
677708,455,677708,128,UEFA EL,UEFA Europa League,U,4,International,G,1053,PSV,2314,LASK Linz,-0.50,2.75,-0.25,1.00,2.0,0.0,0.0,1.0,0.0,0.0,33.0,42.0,6.0,6.0,3.0,0.0,2.0,1.0,62.0,38.0,21.0,0.0
677709,416,677709,128,UEFA EL,UEFA Europa League,U,4,International,G,5682,Celtic,6565,Lazio,-0.25,2.50,0.00,1.00,1.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0,6.0,2.0,0.0,0.0,1.0,0.0,53.0,47.0,7.0,0.0
677711,494,677711,128,UEFA EL,UEFA Europa League,U,4,International,G,984,Sevilla,6348,F91 Dudelange,-3.25,4.25,-1.50,2.00,5.0,1.0,0.0,1.0,0.0,0.0,84.0,24.0,28.0,8.0,3.0,0.0,7.0,1.0,73.0,27.0,36.0,0.0
677712,415,677712,128,UEFA EL,UEFA Europa League,U,4,International,G,6382,Eintracht Frankfurt,1127,Standard Liege,-1.50,3.25,-0.75,1.50,1.0,0.0,0.0,0.0,0.0,0.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,69.0,31.0,7.0,0.0
677713,414,677713,128,UEFA EL,UEFA Europa League,U,4,International,G,889,Trabzonspor,239,Krasnodar,-0.25,2.75,0.00,1.25,0.0,2.0,0.0,0.0,0.0,0.0,5.0,13.0,3.0,4.0,1.0,0.0,0.0,0.0,29.0,71.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692712,359892,692712,234,SVK YL,Slovakia Youth League,S,28,Slovakia,S,1567,MFK Zemplin Michalovce U19,2575,MFK Ruzomberok U19,0.50,2.50,0.25,1.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0,35.0,3.0,0.0
692839,359129,692839,881,AFC C U19,AFC U19 Championships,A,4,International,G,15950,Malaysia U19,15920,Brunei U19,-6.00,6.75,-2.50,2.75,6.0,0.0,0.0,0.0,0.0,0.0,34.0,3.0,36.0,1.0,1.0,0.0,2.0,0.0,94.0,6.0,14.0,0.0
692841,359756,692841,881,AFC C U19,AFC U19 Championships,A,4,International,G,7718,Vietnam U19,28777,Mongolia U19,-4.50,5.25,-2.00,2.25,2.0,0.0,0.0,0.0,0.0,0.0,44.0,38.0,25.0,20.0,2.0,2.0,4.0,3.0,62.0,38.0,32.0,0.0


In [12]:
len(train0)

2192

In [0]:
train1 = train[(train['Status'] <= 45) & (train["Goal"]==1)]
train1 = train1.sort_values("Status").groupby("MatchID", as_index=False).first()

In [14]:
train1.sort_values("Goal", ascending = False).iloc[:,-20:]

,HostHandicapHT,ExpectedGoalsHT,HostCornersFT,GuestCornersFT,HostYCFT,GuestYCFT,HostRCFT,GuestRCFT,HostAttacksFT,GuestAttacksFT,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
0,-0.25,1.00,0.0,0.0,0.0,0.0,0.0,0.0,7.0,19.0,0.0,0.0,0.0,1.0,0.0,1.0,36.0,64.0,6.0,1.0
1059,0.00,0.75,0.0,1.0,0.0,0.0,0.0,0.0,5.0,4.0,4.0,3.0,2.0,0.0,0.0,0.0,84.0,16.0,7.0,1.0
1057,0.50,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,97.0,12.0,1.0
1056,0.50,1.50,0.0,0.0,0.0,1.0,0.0,0.0,14.0,17.0,3.0,7.0,0.0,1.0,1.0,0.0,36.0,64.0,17.0,1.0
1055,-0.25,1.00,3.0,0.0,0.0,1.0,0.0,0.0,39.0,34.0,23.0,9.0,2.0,1.0,3.0,1.0,58.0,42.0,37.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,-0.25,1.00,2.0,0.0,0.0,0.0,0.0,0.0,7.0,3.0,6.0,1.0,1.0,1.0,1.0,0.0,83.0,17.0,6.0,1.0
523,-0.25,1.00,4.0,2.0,0.0,0.0,0.0,0.0,28.0,23.0,14.0,11.0,3.0,2.0,4.0,5.0,59.0,41.0,30.0,1.0
522,-0.25,1.00,3.0,0.0,1.0,0.0,0.0,0.0,38.0,29.0,29.0,15.0,1.0,0.0,0.0,1.0,57.0,43.0,24.0,1.0
521,0.75,1.75,0.0,1.0,0.0,0.0,0.0,0.0,9.0,4.0,2.0,3.0,1.0,0.0,0.0,0.0,62.0,38.0,6.0,1.0


In [15]:
len(train1)

1578

In [16]:
len(train1.MatchID.unique())

1578

In [0]:
ntrain = train1.append(train0)
ntrain1 = ntrain.append(train00)
ntrain2 = ntrain.append(train000)
ntrain1 = ntrain1.drop_duplicates(subset="MatchID",keep='first')
ntrain2 = ntrain1.drop_duplicates(subset="MatchID",keep='first')
ntrain1 = ntrain1.dropna()
ntrain2 = ntrain2.dropna()


In [18]:
ntrain

,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,GuestTeamID,GuestTeamName,HostHandicapFT,ExpectedGoalsFT,HostHandicapHT,ExpectedGoalsHT,HostCornersFT,GuestCornersFT,HostYCFT,GuestYCFT,HostRCFT,GuestRCFT,HostAttacksFT,GuestAttacksFT,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
0,677710,128,UEFA EL,UEFA Europa League,U,4,International,G,5378,Dynamo Kiev,5646,FC Copenhagen,-0.50,2.25,-0.25,1.00,0.0,0.0,0.0,0.0,0.0,0.0,7.0,19.0,0.0,0.0,0.0,1.0,0.0,1.0,36.0,64.0,6.0,1.0
1,677712,128,UEFA EL,UEFA Europa League,U,4,International,G,6382,Eintracht Frankfurt,1127,Standard Liege,-1.50,3.25,-0.75,1.50,2.0,0.0,0.0,0.0,0.0,0.0,37.0,37.0,21.0,11.0,2.0,0.0,3.0,1.0,62.0,38.0,37.0,1.0
2,677714,128,UEFA EL,UEFA Europa League,U,4,International,G,6693,Getafe,1134,Basel,-0.75,2.50,-0.25,1.00,1.0,0.0,0.0,0.0,0.0,0.0,30.0,21.0,8.0,4.0,0.0,2.0,2.0,2.0,50.0,50.0,22.0,1.0
3,677716,128,UEFA EL,UEFA Europa League,U,4,International,G,6493,Arsenal,6323,Guimaraes,-1.50,3.00,-0.75,1.25,1.0,1.0,0.0,0.0,0.0,0.0,26.0,27.0,11.0,12.0,0.0,2.0,2.0,1.0,53.0,47.0,21.0,1.0
4,677717,128,UEFA EL,UEFA Europa League,U,4,International,G,1058,Rennes,1629,CFR Cluj,-0.50,2.25,-0.25,0.75,0.0,1.0,1.0,1.0,1.0,0.0,26.0,15.0,16.0,6.0,1.0,1.0,1.0,1.0,52.0,48.0,21.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(692712, 359892)",692712,234,SVK YL,Slovakia Youth League,S,28,Slovakia,S,1567,MFK Zemplin Michalovce U19,2575,MFK Ruzomberok U19,0.50,2.50,0.25,1.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0,35.0,3.0,0.0
"(692839, 359129)",692839,881,AFC C U19,AFC U19 Championships,A,4,International,G,15950,Malaysia U19,15920,Brunei U19,-6.00,6.75,-2.50,2.75,6.0,0.0,0.0,0.0,0.0,0.0,34.0,3.0,36.0,1.0,1.0,0.0,2.0,0.0,94.0,6.0,14.0,0.0
"(692841, 359756)",692841,881,AFC C U19,AFC U19 Championships,A,4,International,G,7718,Vietnam U19,28777,Mongolia U19,-4.50,5.25,-2.00,2.25,2.0,0.0,0.0,0.0,0.0,0.0,44.0,38.0,25.0,20.0,2.0,2.0,4.0,3.0,62.0,38.0,32.0,0.0
"(693014, 359425)",693014,1450,BRA Cup U20,Brazil U20 Cup,B,47,Brazil,B,8083,Goias U20,18897,Goiania U20,-1.25,3.50,-0.50,1.50,1.0,1.0,1.0,0.0,0.0,0.0,32.0,18.0,19.0,14.0,4.0,2.0,0.0,1.0,29.0,71.0,22.0,0.0


In [19]:
ntrain.isna().sum()

MatchID                0
LeagueID               0
LeagueName             0
LeagueFullname         0
LeagueAbbreviation     0
CountryID              0
CountryName            0
CountryAbbreviation    0
HomeTeamID             0
HomeTeamName           0
GuestTeamID            0
GuestTeamName          0
HostHandicapFT         3
ExpectedGoalsFT        3
HostHandicapHT         7
ExpectedGoalsHT        7
HostCornersFT          0
GuestCornersFT         0
HostYCFT               0
GuestYCFT              0
HostRCFT               0
GuestRCFT              0
HostAttacksFT          0
GuestAttacksFT         0
HostDAttacksFT         0
GuestDAttacksFT        0
HostonTargetFT         0
GuestonTargetFT        0
HostoffTargetFT        0
GuestoffTargetFT       0
HostPossessionFT       0
GuestPossessionFT      0
Status                 0
Goal                   0
dtype: int64

In [0]:
ntrain = ntrain.dropna()
ntrain = ntrain.drop_duplicates(subset="MatchID",keep='first')

In [0]:
#Oversampling
ntrain = ntrain.append(ntrain1[ntrain1["Goal"]==0])
ntrain = ntrain.append(ntrain2[ntrain2["Goal"]==0])

In [0]:
# Indicies of each class' observations
random.seed(s)
i_class0 = np.where(ntrain["Goal"] == 1)[0]
i_class1 = np.where(ntrain["Goal"] == 0)[0]

#Get size of the underrepresented class
n_class0 = len(i_class0)

i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)
ntrain = ntrain.iloc[i_class0,:].append(ntrain.iloc[i_class1_downsampled,:])

In [23]:
ntrain.sort_values(by = "Goal", ascending = False)

,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,GuestTeamID,GuestTeamName,HostHandicapFT,ExpectedGoalsFT,HostHandicapHT,ExpectedGoalsHT,HostCornersFT,GuestCornersFT,HostYCFT,GuestYCFT,HostRCFT,GuestRCFT,HostAttacksFT,GuestAttacksFT,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
0,677710,128,UEFA EL,UEFA Europa League,U,4,International,G,5378,Dynamo Kiev,5646,FC Copenhagen,-0.50,2.25,-0.25,1.00,0.0,0.0,0.0,0.0,0.0,0.0,7.0,19.0,0.0,0.0,0.0,1.0,0.0,1.0,36.0,64.0,6.0,1.0
1059,688606,1197,ZMB SL,Zambia Super League,Z,179,Zambia,Z,4655,Nkana FC,15213,Red Arrows FC,0.00,2.00,0.00,0.75,0.0,1.0,0.0,0.0,0.0,0.0,5.0,4.0,4.0,3.0,2.0,0.0,0.0,0.0,84.0,16.0,7.0,1.0
1057,688604,1197,ZMB SL,Zambia Super League,Z,179,Zambia,Z,30295,Buildcon FC,15112,NAPSA Stars,0.00,2.00,0.50,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,97.0,12.0,1.0
1056,688595,917,AFC C U19W,AFC U19 Womens Championship,A,4,International,G,7762,Thailand Women U19,10274,Australia Women U19,1.00,3.50,0.50,1.50,0.0,0.0,0.0,1.0,0.0,0.0,14.0,17.0,3.0,7.0,0.0,1.0,1.0,0.0,36.0,64.0,17.0,1.0
1055,688588,748,VEN Cup,Venezuela Cup,V,116,Venezuela,W,1832,Deportivo La Guaira,9970,Monagas,-0.50,2.25,-0.25,1.00,3.0,0.0,0.0,1.0,0.0,0.0,39.0,34.0,23.0,9.0,2.0,1.0,3.0,1.0,58.0,42.0,37.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(692655, 359966)",692655,350,HUN NB2,Hungary NB II,H,44,Hungary,X,7272,Dorogi FC,2293,Bekescsabai Elore SE,-0.50,2.25,-0.25,0.75,0.0,1.0,0.0,0.0,0.0,0.0,6.0,14.0,6.0,12.0,0.0,1.0,0.0,0.0,34.0,66.0,8.0,0.0
"(684696, 96472)",684696,2,CHN SL,China Super League,C,1,China,Z,19561,Jiangsu Suning FC,1211,Shandong Luneng,-0.50,3.00,-0.25,1.25,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,47.0,2.0,0.0
"(687948, 295385)",687948,108,PRT PL,Portugal Primeira Liga,P,45,Portugal,P,6087,Gil Vicente,6207,Maritimo,-0.50,2.00,-0.25,0.75,0.0,1.0,0.0,0.0,0.0,0.0,6.0,5.0,3.0,5.0,0.0,0.0,1.0,2.0,61.0,39.0,6.0,0.0
"(686611, 39664)",686611,721,HRV H2,Croatia 2.HNL,C,98,Croatia,K,19800,Kustosija,14194,Bijelo Brdo,-0.25,2.00,-0.25,0.75,0.0,0.0,0.0,0.0,0.0,0.0,12.0,6.0,6.0,2.0,0.0,0.0,2.0,0.0,56.0,44.0,9.0,0.0


In [24]:
px.histogram(ntrain[ntrain["Goal"]== 0], x = "Status", nbins = 45)

In [25]:
px.histogram(ntrain[ntrain["Goal"]== 1], x = "Status", nbins = 45)

In [26]:
px.histogram(ntrain, x = "Goal")

# Feature Engineering

In [0]:
X = ntrain.iloc[:,12:33]
# stand = MinMaxScaler().fit(X)
# stdX = pd.DataFrame(stand.transform(X))
# pca = PCA(0.99)
# pca.fit(stdX)
# pcX = pca.transform(stdX)
y = ntrain["Goal"]
#Transform variables to numeric
for i in X.columns:
    X[i] = pd.to_numeric(X[i])
y = pd.to_numeric(y)



In [0]:
random.seed(s)
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.7)

In [29]:
#Random Forest
random.seed(s)
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [30]:
# Logistic Regression
random.seed(s)
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
#XGBTree 
random.seed(s)
# cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
# ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
#              'objective': 'binary:logistic'}
# xgb = GridSearchCV(xgb.XGBClassifier(**ind_params), 
#                             cv_params, 
#                              scoring = 'accuracy', cv = 5, n_jobs = -1)
xgb = xgb.XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [32]:
#Neuronal network
tf.keras.backend.clear_session() # For easy reset of notebook state.
#### Ignacio: You MUST explain a bit your neural network topology: 
#### One hidden layer with 128 neurons with the "Relux" activation function 
#### which is better for computational efficiency. Then you add an output layer with 3 neurons (one for each building you want to predict).
#### As the output neurons give you some value between 0-1, you apply the softmax function in order to normalize those outputs and interpret them 
#### as probabilities of being in each building.
nmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

#### Ignacio: Why this choices of optimizer and loss function and metrics?
nmodel.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
nmodel.fit(np.array(X_train), np.array(y_train), epochs=10)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 2206 samples
Epoch 1/10
2206/2206 [==============================] - 0s 70us/sample - loss: 1.4455 - acc: 0.5227
Epoch 2/10
2206/2206 [==============================] - 0s 43us/sample - loss: 0.7043 - acc: 0.6043
Epoch 3/10
2206/2206 [==============================] - 0s 46us/sample - loss: 0.6582 - acc: 0.6319
Epoch 4/10
2206/2206 [==============================] - 0s 50us/sample - loss: 0.7018 - acc: 0.6179
Epoch 5/10
2206/2206 [==============================] - 0s 47us/sample - loss: 0.6230 - acc: 0.6759
Epoch 6/10
2206/2206 [==============================] - 0s 48us/sample - loss: 0.6109 - acc: 0.6895
Epoch 7/10
2206/2206 [==============================] - 0s 50us/sample - loss: 0.6012 - acc: 0.6809
Epoch 8/10
2206/2206 [==============================] - 0s 46us/sample - loss: 0.6372 - acc: 0.6655
Epoch 9/10
2206/2206 [==============================] - 0s 45us/sample - loss: 0.6196 - acc: 0.67

In [33]:
test_loss, test_acc = nmodel.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

946/946 - 0s - loss: 0.6813 - acc: 0.6638

Test accuracy: 0.6638478


In [0]:
#Predictions
rfpreds = rf.predict(X_test)
lrpreds = lr.predict(X_test)
xgbpreds = xgb.predict(X_test)
npreds = np.round(nmodel.predict(X_test)[:,1])

In [0]:
#Probabilities
rfprobs = rf.predict_proba(X_test)[:,1]
lrprobs = lr.predict_proba(X_test)[:,1]
xgbprobs = xgb.predict_proba(X_test)[:,1]
nprobs = nmodel.predict(X_test)[:,1]

In [0]:
#Raül probs
Vscore2 = X_test['HostCornersFT']*0.9+X_test['GuestCornersFT']*0.9+X_test['HostYCFT']+X_test['GuestYCFT'] 
+ X_test['HostRCFT']*-2.0+X_test['GuestRCFT']*-2.0+X_test['HostonTargetFT']*9.0+X_test['GuestonTargetFT']*9.0 
+ X_test['HostoffTargetFT']*5.0+X_test['GuestoffTargetFT']*5.0+X_test['HostAttacksFT']*0.7 
+ X_test['GuestAttacksFT']*0.7+X_test['HostDAttacksFT']+X_test['GuestDAttacksFT']
Vscore2
Vscore2 = Vscore2/X_test['Status']
raulprobs = Vscore2 * 10
raulpreds = round(raulprobs)

In [37]:
raulpreds = raulpreds.replace(np.r_[2:30],1)
raulpreds.unique()

array([1., 0.])

In [0]:
# importances = rf.feature_importances_
# cols = X.columns
# pd.DataFrame({"Variable": cols,"Importance" : importances})

In [0]:
results = pd.DataFrame({"Goal": y_test,"rfpreds" : rfpreds,"lrpreds": lrpreds, "xgbpreds": xgbpreds,
              "rfprobs": rfprobs, "lrprobs": lrprobs, "xgbprobs": xgbprobs, "npreds": npreds, "nprobs": nprobs, "raulpreds": raulpreds, "raulprobs": raulprobs})

In [40]:
results

,Goal,rfpreds,lrpreds,xgbpreds,rfprobs,lrprobs,xgbprobs,npreds,nprobs,raulpreds,raulprobs
1131,1.0,1.0,1.0,1.0,0.825,0.865072,0.866020,1.0,0.852102,1.0,1.500000
"(692435, 356353)",0.0,1.0,0.0,0.0,0.570,0.311348,0.402823,0.0,0.102209,1.0,1.068182
768,1.0,1.0,1.0,1.0,0.770,0.882119,0.776771,1.0,0.804674,1.0,2.064516
"(687774, 161119)",0.0,0.0,0.0,0.0,0.110,0.256986,0.059266,0.0,0.224220,1.0,0.600000
"(689246, 219983)",0.0,0.0,0.0,0.0,0.130,0.469191,0.460399,0.0,0.090769,1.0,1.200000
...,...,...,...,...,...,...,...,...,...,...,...
1527,1.0,1.0,1.0,1.0,0.860,0.650620,0.829502,1.0,0.824777,1.0,0.750000
"(686877, 11860)",0.0,0.0,1.0,0.0,0.080,0.791042,0.478098,1.0,0.502534,1.0,0.760000
1190,1.0,1.0,1.0,1.0,0.725,0.758839,0.636538,0.0,0.373422,1.0,1.350000
144,1.0,1.0,0.0,1.0,0.680,0.371536,0.703497,0.0,0.311581,0.0,0.000000


In [41]:
rf_test = results[results["rfprobs"] >= 0.9]
lr_test = results[results["rfprobs"] >= 0.9]
xgb_test = results[results["raulprobs"] >= 0.9]
n_test = results[results["nprobs"] >= 0.9]
raul_test = results[results["raulprobs"] >= 0.9]
pd.DataFrame({"":["0.9 Accuracy","Accuracy","% 0.9 preds"],
              "rf": [accuracy_score(rf_test["Goal"],rf_test["rfpreds"]), 
                                 accuracy_score(rfpreds,y_test), len(results[results["rfprobs"] >= 0.9])*100/len(X_test)],
              "lr": [accuracy_score(lr_test["Goal"],lr_test["lrpreds"]), 
                                 accuracy_score(lrpreds,y_test), len(results[results["lrprobs"] >= 0.9])*100/len(X_test)],
              "xgb": [accuracy_score(xgb_test["Goal"],xgb_test["xgbpreds"]), 
                                 accuracy_score(xgbpreds,y_test), len(results[results["xgbprobs"] >= 0.9])*100/len(X_test)],
              "nn": [accuracy_score(n_test["Goal"],n_test["npreds"]), 
                                 accuracy_score(npreds,y_test), len(results[results["nprobs"] >= 0.9])*100/len(X_test)],
              "raul": [accuracy_score(raul_test["Goal"],raul_test["raulpreds"]), 
                                 accuracy_score(raulpreds,y_test), len(results[results["raulprobs"] >= 0.9])*100/len(X_test)]
               })



,,rf,lr,xgb,nn,raul
0,0.9 Accuracy,1.000000,0.942857,0.721774,0.842105,0.552419
1,Accuracy,0.849894,0.692389,0.751586,0.663848,0.521142
2,% 0.9 preds,3.699789,2.854123,3.594080,2.008457,52.431290


In [42]:
confusion_matrix(xgbpreds,y_test)

array([[310,  94],
       [141, 401]])

In [43]:
confusion_matrix(lrpreds,y_test)


array([[324, 164],
       [127, 331]])

In [44]:
confusion_matrix(rfpreds,y_test)

array([[389,  80],
       [ 62, 415]])

In [45]:
confusion_matrix(npreds,y_test)

array([[408, 275],
       [ 43, 220]])

In [46]:
confusion_matrix(raulpreds,y_test)

array([[138, 140],
       [313, 355]])

In [47]:
print("Metric   "," RF "," LR ","XGB","NN")
print("Accuracy ", round(accuracy_score(rfpreds,y_test),2), round(accuracy_score(lrpreds,y_test),2), round(accuracy_score(xgbpreds,y_test),2),round(accuracy_score(npreds, y_test),2))
print("Kappa    ",round(cohen_kappa_score(rfpreds,y_test),2),round(cohen_kappa_score(lrpreds,y_test),2),round(cohen_kappa_score(xgbpreds,y_test),2),round(cohen_kappa_score(npreds, y_test),2))
print("Precision",round(precision_score(rfpreds,y_test),2),round(precision_score(lrpreds,y_test),2),round(precision_score(xgbpreds,y_test),2),round(precision_score(npreds, y_test),2))
print("Recall   ", round(recall_score(rfpreds,y_test),2),round(recall_score(lrpreds,y_test),2),round(recall_score(xgbpreds,y_test),2),round(recall_score(npreds, y_test),2))

Metric     RF   LR  XGB NN
Accuracy  0.85 0.69 0.75 0.66
Kappa     0.7 0.39 0.5 0.34
Precision 0.84 0.67 0.81 0.44
Recall    0.87 0.72 0.74 0.84


# Predictions Live

In [0]:
# filename = datetime.now().strftime('%Y-%m-%d/gambling-%Y-%m-%d-%H-%M.csv')
# test = pd.read_csv("/Users/jojoel/Google Drive/Github/bet369/Datasets/"+filename, header = 0)
# del test["BettingCuoteHT"]
# del test["BettingCuoteFT"]
# del test["HostGoalsFT"]
# del test["GuestGoalsFT"]
# test = test.dropna()
# test = test.loc[(test['HostPossessionFT'] != 0) & (test['GuestPossessionFT'] != 0)]
# test = test.drop(test[test["Status"] == "HT"].index)
# test = test.drop(test[test["Status"] == "NS"].index)
# test = test.drop(test[test["Status"] == "FT"].index)
# ntest = test.iloc[:,12:32]
# pctest = pca.transform(ntest)
# tpreds = rf.predict(pctest)
# tprobs = rf.predict_proba(pctest)[:,1]
# test["Prediction"] = (np.round(xgb.predict(pctest)))
# test["Probability"] = rf.predict_proba(pctest)[:,1]
# test["Status"] = pd.to_numeric(test["Status"])
# test0 = test[(test["Status"] <= 45) & (test["Goal"] == 0)]
# test0 = test0[["HomeTeamName","GuestTeamName","Status","Goal","Prediction","Probability","HostonTargetFT","GuestonTargetFT","HostCornersFT","GuestCornersFT",
#        "HostYCFT","GuestYCFT","HostRCFT",
#       "GuestRCFT","HostAttacksFT","GuestAttacksFT","HostoffTargetFT",
#       "GuestoffTargetFT","HostPossessionFT","GuestPossessionFT"]]

# test0.sort_values("Probability", ascending = False)


In [0]:
#Independent of time/Status
#Class balanced
#Less observations